In [2]:
from tqdm import tqdm
from pandarallel import pandarallel
from nltk.corpus import stopwords
from gensim import models
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import ast
import spacy

tqdm.pandas()

In [3]:
with open('paper_selectors.txt', 'r') as file:
    papers = ast.literal_eval(file.read())

with open('party_selectors.txt', 'r') as file:
    parties = ast.literal_eval(file.read())

with open('party_synonyms.txt', 'r') as file:
    party_synonyms = ast.literal_eval(file.read())

with open('months.txt', 'r') as file:
    months = ast.literal_eval(file.read())

with open('min_mentions_per_article.txt', 'r') as file:
    min_val = int(file.read())

for synonym in party_synonyms:
    parties = [x for x in parties if x not in synonym[1:]]

In [4]:
df = pd.read_csv("../../data/06_preprocessed_data.csv")

In [5]:
df.reindex(index=df['Unnamed: 0'])

,Unnamed: 0,Newspaper,Title,Date,Author,Section,Body,VVD,CDA,D66,...,body_vader_scores,body_neg,body_neu,body_pos,body_compound,title_vader_scores,title_neg,title_neu,title_pos,title_compound
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,0,AD,omtzigt mengt strijd boek vol plannen drie ton...,2023-08-21,Niels Klaassen,NaN,pieter omtzigt mikt gematigde groei nieuwe par...,2,5,0,...,"{'neg': 0.0, 'neu': 0.935, 'pos': 0.065, 'comp...",0.000,0.935,0.065,0.4019,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
1,1,Parool,plofpartij gamechanger we verwachten nieuwe pa...,2023-08-21,Niels Klaassen,NaN,pieter omtzigt mikt gematigde groei partij nie...,2,6,0,...,"{'neg': 0.046, 'neu': 0.833, 'pos': 0.121, 'co...",0.046,0.833,0.121,0.9950,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
2,2,Volkskrant,kaarten rechts opnieuw geschud,2023-08-21,RAOUL DU PRÉ,Ten Eerste,analyse vvd sluit pvv frans weisglas oudvoorzi...,15,2,2,...,"{'neg': 0.055, 'neu': 0.823, 'pos': 0.123, 'co...",0.055,0.823,0.123,0.9955,"{'neg': 0.121, 'neu': 0.823, 'pos': 0.056, 'co...",0.121,0.823,0.056,-0.3612
3,3,Trouw,pvv,2023-08-21,STEVO AKKERMAN,Vandaag,dilan yesilgöz opwierp opvolger mark rutte vi...,6,0,0,...,"{'neg': 0.136, 'neu': 0.808, 'pos': 0.056, 'co...",0.136,0.808,0.056,-0.9788,"{'neg': 0.402, 'neu': 0.598, 'pos': 0.0, 'comp...",0.402,0.598,0.000,-0.8126
4,4,AD,vvdkamerlid woordvoerder sport rudmer heerema ...,2023-08-21,Politieke redactie,NaN,vvdkamerlid rudmer heerema keert aankomende tw...,4,1,1,...,"{'neg': 0.081, 'neu': 0.757, 'pos': 0.161, 'co...",0.081,0.757,0.161,0.9943,"{'neg': 0.0, 'neu': 0.69, 'pos': 0.31, 'compou...",0.000,0.690,0.310,0.2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6764,6764,Trouw,vervuiling tast onze bestaanszekerheid,2023-11-27,SIJAS AKKERMAN,Opinie,leefomgeving nieuw sociaal contract nsc partij...,0,0,0,...,"{'neg': 0.004, 'neu': 0.922, 'pos': 0.075, 'co...",0.004,0.922,0.075,0.9677,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
6765,6765,AD,pieter omtzigt veronrustend rapport vuurwerkra...,2023-12-01,Lucien Baard,NaN,„dit gebeurt wanneer regering stukken achterho...,0,0,0,...,"{'neg': 0.123, 'neu': 0.774, 'pos': 0.103, 'co...",0.123,0.774,0.103,-0.9460,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
6766,6766,Volkskrant,pensioenen stijgen verschillen groot zorg verk...,2023-12-01,Hessel von Piekartz,NaN,pensioengerechtigden zien pensioen volgend jaa...,0,0,0,...,"{'neg': 0.08, 'neu': 0.809, 'pos': 0.112, 'com...",0.080,0.809,0.112,0.9903,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000


In [6]:
data = df['Date'].dropna().unique()

In [7]:
vectorizer = CountVectorizer(strip_accents='unicode', lowercase=False)
content = df['Title'].dropna()
data_vectorized = vectorizer.fit_transform(tqdm(content))

100%|██████████| 6765/6765 [00:00<00:00, 191254.16it/s]

In [8]:
stop_words = stopwords.words('dutch') + [x.lower() for x in parties] + ['wilders', 'omtzigt', 'yesilgoz', 'timmermans', 'frans', 'pieter', 'caroline', 'ouwehand', 'baudet', 'marijnissen', 'hoekstra', 'glpvda']
vectorizer = TfidfVectorizer(strip_accents='unicode', lowercase=False, stop_words=stop_words)
content = df['Title'].dropna()
data_vectorized = vectorizer.fit_transform(tqdm(content))

  0%|          | 0/6765 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['groenlinks', 'pvda'] not in stop_words.
  warnings.warn(
100%|██████████| 6765/6765 [00:00<00:00, 208583.64it/s]


In [9]:
# Define Search Param
search_params = {'n_components': [i for i in range(5,15)], 'learning_decay': np.arange(0.5,1,0.1)}
# Init the Model
lda = LatentDirichletAllocation(random_state=42)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, verbose=3, n_jobs=-1)

# Do the Grid Search
lda_topics = model.fit(data_vectorized)
model.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 2/5] END learning_decay=0.5, n_components=5;, score=-47088.541 total time=   3.5s
[CV 1/5] END learning_decay=0.5, n_components=6;, score=-53422.014 total time=   3.3s
[CV 1/5] END learning_decay=0.5, n_components=5;, score=-50165.463 total time=   3.6s
[CV 2/5] END learning_decay=0.5, n_components=6;, score=-50254.142 total time=   3.6s
[CV 3/5] END learning_decay=0.5, n_components=5;, score=-50619.175 total time=   3.7s
[CV 3/5] END learning_decay=0.5, n_components=6;, score=-53891.484 total time=   3.6s
[CV 5/5] END learning_decay=0.5, n_components=5;, score=-49823.004 total time=   3.7s
[CV 4/5] END learning_decay=0.5, n_components=5;, score=-51439.968 total time=   4.0s


KeyboardInterrupt: 

In [10]:
stop_words = stopwords.words('dutch') + [x.lower() for x in parties] + ['groenlinkspvda','pvdagl', 'geert','keijzer', 'mona', 'wilders', 'omtzigt', 'yesilgoz', 'timmermans', 'frans', 'pieter', 'caroline', 'ouwehand', 'baudet', 'marijnissen', 'hoekstra', 'glpvda', 'wopke']
vectorizer = TfidfVectorizer(strip_accents='unicode', lowercase=False, stop_words=stop_words)
lda_model = LatentDirichletAllocation(random_state=42, n_components=20)

for date in data:
    content = df[df['Date'] == '2023-11-22']['Title'].dropna()
    data_vectorized = vectorizer.fit_transform(tqdm(content))

    lda_topics = lda_model.fit(data_vectorized)

100%|██████████| 78/78 [00:00<00:00, 159822.04it/s]


In [ ]:
def print_top_words(lda: pd.DataFrame, vectorizer: pd.DataFrame, n_top_words: int=20, n_topics: int=10) -> None:
    """Print the top words for each topic.
     
     Args:
        topic_modelling_files (dict[str, dict[str, pd.DataFrame]]): Dictionary containing topic modelling data.
        n_top_words (int): Number of words printed per topic. Defaults to 10.
        n_topics (int): Number of topics to print. Defaults to 3.
    """
    feature_names = vectorizer.get_feature_names_out()
    dist = list(enumerate(lda.components_))

    for topic_id, topic in dist[:n_topics]:
        message = f"Topic {topic_id}: "
        message += " ".join([feature_names[i] for i in topic.argsort()[: -n_top_words - 1 : -1]])
        print(message)
    print()

moet meer stopwoorden opschonen (politieke partijen, partijleiders)

In [11]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

stop_words = stopwords.words('dutch') + [x.lower() for x in parties] + [
    'groenlinkspvda','pvdagl', 'geert','keijzer', 'mona', 'wilders', 'omtzigt', 'yesilgoz', 'timmermans', 
    'frans', 'pieter', 'caroline', 'ouwehand', 'baudet', 'marijnissen', 'hoekstra', 'glpvda', 'wopke', 'we',
    'heerema', 'vvder'
    ]

stop_words = []
vectorizer = TfidfVectorizer(strip_accents='unicode', lowercase=False, stop_words=stop_words)
lda_model = LatentDirichletAllocation(random_state=42, n_components=5)

df_topics_title = pd.DataFrame()

for date in data:
    content = df[df['Date'] == date]['Title'].dropna()
    data_vectorized = vectorizer.fit_transform(tqdm(content))

    lda_topics = lda_model.fit(data_vectorized)

    topics = show_topics(vectorizer=vectorizer, lda_model=lda_topics, n_words=5)

    df_topics_title[date] = topics

100%|██████████| 115/115 [00:00<00:00, 178514.05it/s]
/var/folders/9z/gq738z712wv0tz09r4b4vw6c0000gn/T/ipykernel_639/2806248588.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_topics_title[date] = topics
100%|██████████| 102/102 [00:00<00:00, 168631.85it/s]
/var/folders/9z/gq738z712wv0tz09r4b4vw6c0000gn/T/ipykernel_639/2806248588.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_topics_title[date] = topics
100%|██████████| 104/104 [00:00<00:00, 177826.18it/s]
/var/folders/9z/gq738z712wv0tz09r4b4vw6c0000gn/T/ipykern

In [12]:
df_topics_title.T

,0,1,2,3,4
2023-08-21,"[partij, nieuwe, omtzigt, pieter, plofpartij]","[pvv, politieke, omtzigts, pieter, partij]","[verzekerd, oplossing, fiets, wel, opnieuw]","[omtzigt, jeugd, dienstplicht, lanterfantende,...","[politiek, hijum, stapt, ruim, we]"
2023-08-22,"[omtzigts, timmermans, partij, omtzigt, links]","[impact, nsc, hooligan, logo, aansluiten]","[niks, partij, dierenwelzijn, vvdkamerlid, kamer]","[omtzigt, pieter, ineens, timmermans, blijft]","[pieter, omtzigt, veren, ideologische, cruciale]"
2023-08-23,"[tour, personalisme, persoon, college, yesilgoz]","[gundogan, nilufer, bezem, politiek, timmermans]","[zuidplas, biodiversiteit, weinig, bescherming...","[partij, dieren, voelen, verenigd, links]","[timmermans, premier, pleit, volgt, terug]"
2023-08-24,"[d66, wethouder, geervliet, opheldering, eist]","[sp, pvdd, schatkist, genoeg, daklozenprobleem]","[vera, bergkamp, nooit, pieter, kamervoorzitter]","[stopt, weekendtips, vleermuizen, doetinchem, ...","[hoekstra, timmermans, eurocommissaris, wopke,..."
2023-08-25,"[hoekstra, bestuursakkoord, opvolger, brussel,...","[staaij, timmermans, week, coalitie, presentatie]","[brieven, coalitieakkoord, tweede, dag, eerste]","[verf, kwam, voorzitter, exit, zit]","[sgpleider, staaij, 25, jaar, europarlement]"
...,...,...,...,...,...
2023-12-02,"[nederland, gaat, 6632, kruiswoordtest, komt]","[lessen, vs, sterke, mangezocht, stad]","[politiek, wilders, krijgen, partij, begrijpt]","[wel, leg, mediacolumn, klas, koorddansen]","[gezicht, kat, huis, anders, tweede]"
2023-12-03,"[complotdenken, gevolgd, allemaal, we, vooral]","[pvv, tussen, verkenningsfase, gedoe, dreigt]","[politici, proefcoalitie, begin, stappen, stap]","[kiezersmoraal, luistert, weg, weer, nieuwe]","[laat, keniaanse, geert, wilders, praten]"
2023-12-04,"[nederlanders, alle, wit, zien, vertrouwelijke]","[lezersreactie, vvd, begrepen, nieuwe, pvv]","[partijleiders, plasterk, breed, antiwilderspr...","[vvdlijsttrekker, azmani, europa, ophouden, pa...","[294, woordgraptogram, nr, populisme, mainstream]"
2023-12-05,"[bosma, laat, stoppen, zeker, hangen]","[amsterdam, waarom, en, lhbtiqers, veiligheid]","[griffier, omtzigt, burgemeester, centrumrecht...","[wilders, gemeenteraad, talmen, gelukkig, mees...","[mindaa, kamer, tweede, kwijt, stem]"


In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_topics.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names_out()

# View
df_topic_keywords.head()

,10,100,1000,100000,100jarige,103,105,1050,10de,11,...,zweven,zwevende,zwijgen,zwijgende,zwijgt,zwijndrecht,zwitsal,zwolle,zwols,zwolse
0,0.396251,0.050000,0.462202,0.05,0.414178,0.050000,0.050000,0.05,0.05,0.050000,...,0.05,0.0500,0.05,0.05,0.050000,0.050000,0.05,0.050000,0.05,0.05
1,0.338595,0.389534,0.050000,0.05,0.050000,0.050000,0.050000,0.05,0.05,0.050000,...,0.05,0.5561,0.05,0.05,0.050000,0.050000,0.05,0.361348,0.05,0.05
2,0.050000,0.349302,0.050000,0.05,0.050000,0.050000,0.050000,0.05,0.05,0.050000,...,0.05,0.0500,0.05,0.05,0.384895,0.667636,0.05,0.050000,0.05,0.05
3,0.050000,0.050000,0.050000,0.05,0.050000,0.938082,0.050000,0.05,0.05,0.050000,...,0.05,0.0500,0.05,0.05,0.050000,0.050000,0.05,0.050000,0.05,0.05
4,1.597446,0.050000,0.050000,0.05,0.050000,0.050000,0.394032,0.05,0.05,0.452733,...,0.05,0.0500,0.05,0.05,0.050000,0.050000,0.05,0.398590,0.05,0.05


In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_topics, n_words=15)        

for topic in topic_keywords:
    
display(topic_keywords)

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

[array(['troef', 'kiezers', 'hilversumse', 'wijkcentrum', 'twijfel',
        'verkiezingsfeestje', 'bier', 'gestrooid', 'gratis', 'geld',
        'groen', 'buurtbewoners', 'licht', 'bouwproject', 'bolberg'],
       dtype=object),
 array(['manager', 'torentje', 'weer', 'genoeg', 'manier', 'overheid',
        'krijgt', 'kom', 'haha', 'beveiliging', 'nee', 'nieuwe', 'cultuur',
        'omslag', 'hoeve'], dtype=object),
 array(['kind', 'kiest', 'scholier', 'rechts', 'liever', 'links', 'lokale',
        'speelt', 'expats', 'situatie', 'tien', 'bepalend', 'vele', 'rol',
        'veldhoven'], dtype=object),
 array(['2023', 'chocolade', 'kiezers', 'mensen', 'lessen', 'invloed',
        'peilingen', 'campagne', 'gelooft', 'laatste', 'premierbelofte',
        'tvdebat', 'schieten', 'scherp', 'jonge'], dtype=object),
 array(['ideologisch', 'strategisch', 'stem', 'deur', 'opeens', 'sluit',
        'weer', 'stemmoment', 'ingrijpen', 'mediacircus', 'eigenlijk',
        'rond', 'moeten', 'man', 'loop

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,troef,kiezers,hilversumse,wijkcentrum,twijfel,verkiezingsfeestje,bier,gestrooid,gratis,geld,groen,buurtbewoners,licht,bouwproject,bolberg
Topic 1,manager,torentje,weer,genoeg,manier,overheid,krijgt,kom,haha,beveiliging,nee,nieuwe,cultuur,omslag,hoeve
Topic 2,kind,kiest,scholier,rechts,liever,links,lokale,speelt,expats,situatie,tien,bepalend,vele,rol,veldhoven
Topic 3,2023,chocolade,kiezers,mensen,lessen,invloed,peilingen,campagne,gelooft,laatste,premierbelofte,tvdebat,schieten,scherp,jonge
Topic 4,ideologisch,strategisch,stem,deur,opeens,sluit,weer,stemmoment,ingrijpen,mediacircus,eigenlijk,rond,moeten,man,loop
Topic 5,echt,veranderen,land,wint,huisbaas,huurder,strijd,meerderheid,allerarmste,westlanders,onacceptabel,gemeenteraad,bezuinigen,heel,hoge
Topic 6,uitleg,sorry,briefje,collegas,internationale,adem,houden,regeren,brengt,poffertjeskoningin,rachel,binnen,hoor,lisa,westerveld
Topic 7,bekende,nachtmerrie,roer,komt,twentenaren,klap,aanslag,bierfles,dader,antifabeweging,politieke,sporten,dutje,vorige,deden
Topic 8,geluid,vind,wijk,onwenselijk,spannend,ongemeen,samenwerken,stemt,ai,wispelturig,strijd,gaat,weer,links,volg
Topic 9,klas,bespreken,duo,jij,voorbij,tijd,gaat,rijssenaar,folder,probeert,isa,kahraman,overtuigen,verkiezingsdag,zwevende


In [ ]:
8 //7

1

In [ ]:
week = 0
for i in range(6):
    week += 1
print(week)

6


In [ ]:
week = 1
df_weekly_topics_titles = pd.DataFrame()
series = pd.Series()

for idx, date in enumerate(data):
    if (idx // 7) < week:
        series = pd.concat([series,df[df['Date']==date]['Title']])
        series.reset_index(drop=True, inplace=True)
    else:
        df_weekly_topics_titles[f"week_{idx // 7}"] = series.reindex(range(1019))
        week += 1
        series = pd.Series()
df_weekly_topics_titles

,week_1,week_2,week_3,week_4,week_5,week_6,week_7,week_8,week_9,week_10,week_11,week_12,week_13,week_14,week_15
0,omtzigt mengt strijd boek vol plannen drie ton budget clubje oudcdaers helpers,centrumlinkse coalitie pvdd denk lonkt den haag waar mos buiten boot valt,verschuiving rechts,omstreden spreidingswet verkiezingen behandeld tweede kamer,onderzoek haagse poppetjescarrousel zorgt vertrouwen politiek,den haag weer coalitie d66 groenlinks cda pvda denk pvdd gaan stad besturen,opeens invoering bindend correctief referendum heel dichtbij,tweederde eerste kamer vóór correctief bindend referendum – komt hangt af verkiezingsuitslag,presidentiële pose pieter omtzigt partij verkopen,tweede kamer stemt voorstel koning weer formatie betrekken,lot spreidingswet handen twee senatoren,politieke partijen blijven grenzen recht opzoeken,verkiezingsdebat moerdijkse jongeren zorgen jullie studeren werken dak boven hoofd krijg,denken partijen belangrijkste themas acht minuten,welke politici stemden amsterdammers
1,plofpartij gamechanger we verwachten nieuwe partij pieter omtzigt,asielbeleid splijtzwam komt nieuwe coalitie wel goede aanpak omtzigt nederland bang uitzondering,groenlinkspvda minimumloon 16 euro pleit ovklimaatkaart,omstreden dwangwet verkiezingen behandeld,portemonnee kiezer vullen,centrumlinks bereikt akkoord nieuw gemeentebestuur den haag,meerderheid tweede kamer tóch spreidingswet,referendum komt weer stap dichterbij,thierry baudet forum democratie wij gek wereld gek,kunt omtzigt wel cda halen cda omtzigt,kans gevaar denken politieke partijen kunstmatige intelligentie,onderzoek orde advocaten partijprogrammas botsen rechtsstaat omtzigt,debatmarathon aantocht momenten tvdebatten verkiezingen,amsterdammers kun stemmen tweede kamerverkiezingen 2023,pvv oppermachtig randen stad glpvda centrum stemde rotterdam
2,kaarten rechts opnieuw geschud,botsing lastenverhoging gaan zes partijen onderhandelen centrumlinkse coalitie,groenlinkspvda willen groen sociaal strak koppelen,opinie nieuw sociaal contract bbb middenklasse vaste grond bieden,draaiboek prinsjesdag hoedjesparade alexias debuut troonrede koffertje – daarna twee dagen debat,column goede slechte pensioenideeën verkiezingsprogrammas,presentatie vanhaags akkoord gratis ov kinderen 30 km binnen ring,consensus politiek nooit groot minimumloon omhoog,getouwtrek iser vluchtelingenwerk,elke lijsttrekker móet jaar wel bescheiden,politiek amersfoort botst flink grote problemen lijkt alsof kunt wachten bezuinigen,staat verkiezingsprogrammas landbouw,rtldebat lilian marijnissen pakt kans afwezigheid pieter omtzigt frans timmermans,terug zwevende kiezers erger omtzigt,binnenhof zien premier wilders
3,pvv,boze d66ers maakten yesilgöz bijna premier,pvdagroenlinks minimumloon 16 euro pleit ovklimaatkaart,nieuwe verkenner hellendoorn begint zware klus flinke knauw vertrouwen cda lokaal hellendoorn,ligt open den haag middenpartijen zoeken elkaar,kamer voelt wel opvangdwang,linkse blok vvd zijspoor,column ondanks tijdgeest scoort links steeds slecht kiezer,wegenbelasting noordholland blijft laagste ondanks verhoging,links rechts minderheid omtzigt staat belangrijke keuzes,politieke overpeinzingen,boeren oog natuur opkomst systeem loopt grenzen,flitsbezoeken dries agt joop den uyl kiezers twente,comfort boeren vissers versus belang natuur,wilders zoekt toon én blijft oude
4,vvdkamerlid woordvoerder sport rudmer heerema stopt tweede kamerverkiezingen,vijf themas maken begroting prinsjesdag ingewikkeld,asielzoekers stikstof klimaat tweede kamer besluit welke onderwerpen mag praten,luchtvaart evenredig vluchten inleveren eerste stap krimp definitief tegengehouden vvd,vvd haalt veruit meeste grote donaties binnen partijkas 4 ton laatste weken,eerste kamer stemt thuiswerkwet weg draai vvd bbb meerderheid vindt regelgeving onnodig,kamer miljardenplannen vertimmeren verhoging minimumloon waarschijnlijk uitgesteld,tweede kamer neemt spreidingswet staatssecretaris burg voorlopig,falend huisvestingsbel

In [ ]:
print(df_weekly_topics.keys())

Index(['week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'week_6', 'week_7',
       'week_8', 'week_9', 'week_10', 'week_11', 'week_12', 'week_13',
       'week_14', 'week_15'],
      dtype='object')


In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

[x.lower() for x in parties]+  [
    'groenlinkspvda','pvdagl', 'geert','keijzer', 'mona', 'wilders', 'omtzigt', 'yesilgoz', 'timmermans', 
    'frans', 'pieter', 'caroline', 'ouwehand', 'baudet', 'marijnissen', 'hoekstra', 'glpvda', 'wopke', 'we',
    'heerema', 'vvder', 'nieuw', 'jetten', 'plas', 'kamer', 'tweede'
    ]

stop_words = stopwords.words('dutch') 
vectorizer = TfidfVectorizer(strip_accents='unicode', lowercase=False, stop_words=stop_words)
lda_model = LatentDirichletAllocation(random_state=42, n_components=10)


search_params = {'n_components': [i for i in range(5,15)], 'learning_decay': np.arange(0.5,1,0.1)}
# Init the Model
lda = LatentDirichletAllocation(random_state=42, n_components=10, learning_decay=0.5)

df_topics_title = pd.DataFrame()

for week in df_weekly_topics_titles.keys():
    content = df_weekly_topics_titles[week].dropna()
    data_vectorized = vectorizer.fit_transform(tqdm(content))

    # # Init Grid Search Class
    # model = GridSearchCV(lda, param_grid=search_params, verbose=3, n_jobs=-1)

    # # Do the Grid Search
    # model.fit(data_vectorized)
    # best_model = model.best_estimator_
    # lda_topics = best_model.fit(data_vectorized)

    lda_topics = lda_model.fit(data_vectorized)

    topics = show_topics(vectorizer=vectorizer, lda_model=lda_topics, n_words=10)
    df_topics_title[week] = topics

100%|██████████| 522/522 [00:00<00:00, 257005.13it/s]


In [ ]:
df_topics_title

,week_1,week_2,week_3,week_4,week_5,week_6,week_7,week_8,week_9,week_10,week_11,week_12,week_13,week_14,week_15
0,"[staaij, kees, kondigt, omtzigt, sgpleider, vertrek, nsc, bbb, partij, pieter]","[dienstplicht, weg, den, haag, vvd, nieuw, kiezers, leven, militaire, kanjers]","[lahlah, lijst, nummer, twee, weg, pvdagroenlinks, fatsoen, rechts, mogelijk, wethouder]","[candan, d66lijst, politieke, reacties, yesim, sociaal, nieuw, sanders, bernie, minimum]","[biomassacentrales, provinciepolitiek, nieuwe, klm, d66, dikke, tijdperkrutte, streep, klimaathaast, transgenderverlof]","[geld, migratie, yesilgoz, erfenis, slopen, rutte, 50plus, tekort, wachten, kamer]","[stopt, hoekstra, rebellion, extinction, farizeeer, brief, eupluche, per, boeren, boven]","[cdaer, bontenbal, kamer, fout, we, elkaar, adema, partij, dieren, duivels]","[gek, vvd, omtzigt, yesilgoz, duur, d66, wereld, wij, bokspartij, verbale]","[strategie, premier, rutte, yesilgoz, lang, jaar, omtzigt, gaan, plannen, timmermans]","[vvd, sahla, bolkestein, weg, geld, ontfutselen, stemwijzer, debat, soumaya, vele]","[waar, omtzigt, gaat, kiezer, komt, campagne, kritiek, groen, begroting, kans]","[willen, omtzigt, breda, partij, minder, kabinet, komen, wilders, welke, asielzoekers]","[pvv, zat, wilders, klimaat, mensen, vanwege, wel, azc, overlast, verkiezingswinst]","[liberalen, onzekere, koers, vvd, heel, nieuwe, kamer, mening, bijna, leden]"
1,"[sociaal, contract, nieuw, landelijke, weg, wel, premier, heiligverklaring, past, politicus]","[kabinet, nederland, weer, roomsrood, minister, toe, anders, maakt, mening, adopties]","[campagnetips, mannetjes, betrouwbare, bestaanszekerheid, wethouder, mag, cda, plas, bbb, zelfs]","[lijsttrekker, af, wilders, zelden, partijleider, weer, pvdd, nieuwe, subsidie, goed]","[nieuwe, kamer, af, tekenen, wensen, gaat, verkiezingen, verbondjes, wel, miljarden]","[politici, lijst, omtzigt, excdaers, model, lokale, college, weer, vvd, lijsttrekker]","[blijft, weg, elke, explosies, klacht, jaar, voorzitter, nsc, vrij, blauw]","[bezwaar, natuurvergunning, kabinet, gesprekken, bernhardstraten, infobordjes, eist, krijgt, we, gaan]","[nederland, politieke, kolonialisme, steunt, nsc, presenteert, dinsdag, steun, verkiezingsprogramma, raadswerk]","[partijen, gent, lezing, thierry, geslagen, hoofd, baudet, liberale, beperken, arbeidsmigratie]","[politiek, raad, senaat, ontluisterend, kamer, mobieltje, bbb, dwangwet, asielwet, nieuw]","[voorst, timmermans, rico, verhoeven, nooit, gratis, d66, doet, blij, jaar]","[verdwenen, links, wilders, tweede, kamer, verkiezingscampagne, boeren, thema, concessies, islamstandpunten]","[wilders, pvv, formatiepuzzel, sgp, plannen, hersenkraker, kunst, bbb, blijft, kiezers]","[nodig, staatssecretaris, uslu, wilders, plasterk, corendon, politieke, sp, migratie, d66]"
2,"[omtzigt, partij, regionaal, kiesstelsel, pieter, ziet, nieuw, nieuwe, bescherming, traag]","[eigenlijk, nieuw, nodig, komt, politiek, vrijdag, rond, wilders, vvd, inhoud]","[spreidingswet, euro, omstreden, ovklimaatkaart, minimumloon, 16, gaat, kamer, pleit, personalia]","[ouwehand, esther, bestuur, tour, college, partij, dieren, slag, stok, forum]","[karavaan, los, wachtgeld, waarom, arbeider, patroon, snel, zicht, hard, parlement]","[haag, den, omtzigt, nieuwe, partijen, coalitie, pieter, kandidatenlijst, nsc, doet]","[amerikaanse, dreigen, toestanden, nauwelijks, referendum, laat, verdiend, gaat, stad, eerlijk]","[gemeenten, tweede, lezersreacties, eisen, rectificatie, kamer, straks, leden, bbb, eindhovense]","[tussen, maak, af, oorlog, stelt, moslims, proef, huwelijk, goes, fvd]","[politieke, pechgeneratie, lijsttrekker, elke, bescheiden, bereid, volgt, omtzigt, trekken, nou]","[yesilgoz, pvv, neemt, afstand, bbb, wilders, vast, tijdens, macron, mee]","[nieuwe, omtzigt, partijen, weer, politieke, politici, gek, yesilgoz, wensdenken, kleeft]","[we, timmermans, plan, gaan, doet, waarde, legendarisch, grote, yesilgoz, nsc]","[kamerverkiezingen, tweede, 

In [ ]:
week = 1
df_weekly_topics_body = pd.DataFrame()
series = pd.Series()
for idx, date in enumerate(data):
    if (idx // 7) < week:
        df2 = df[~df.index.duplicated()]
        series = pd.concat([series,df2[df2['Date']==date]['Body']])
        series.reset_index(drop=True, inplace=True)
        bob = series
    else:
        df_weekly_topics_body[f"week_{idx // 7}"] = series.reindex(range(1019))
        week += 1
        series = pd.Series()

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

[x.lower() for x in parties]+  [
    'groenlinkspvda','pvdagl', 'geert','keijzer', 'mona', 'wilders', 'omtzigt', 'yesilgoz', 'timmermans', 
    'frans', 'pieter', 'caroline', 'ouwehand', 'baudet', 'marijnissen', 'hoekstra', 'glpvda', 'wopke', 'we',
    'heerema', 'vvder', 'nieuw', 'jetten', 'plas', 'kamer', 'tweede'
    ]

stop_words = stopwords.words('dutch') 
vectorizer = TfidfVectorizer(strip_accents='unicode', lowercase=False, stop_words=stop_words)
lda_model = LatentDirichletAllocation(random_state=42, n_components=10)

search_params = {'n_components': [i for i in range(5,15)], 'learning_decay': np.arange(0.5,1,0.1)}
# Init the Model
lda = LatentDirichletAllocation(random_state=42, n_components=10, learning_decay=0.5)

df_topics_body = pd.DataFrame()

for week in df_weekly_topics_body.keys():
    content = df_weekly_topics_body[week].dropna()
    data_vectorized = vectorizer.fit_transform(tqdm(content))

    # # Init Grid Search Class
    # model = GridSearchCV(lda, param_grid=search_params, verbose=3, n_jobs=-1)

    # # Do the Grid Search
    # model.fit(data_vectorized)
    # best_model = model.best_estimator_
    # lda_topics = best_model.fit(data_vectorized)

    lda_topics = lda_model.fit(data_vectorized)

    topics = show_topics(vectorizer=vectorizer, lda_model=lda_topics, n_words=10)
    df_topics_body[week] = topics

100%|██████████| 522/522 [00:00<00:00, 3210.63it/s]


In [ ]:
df_topics_body

,week_1,week_2,week_3,week_4,week_5,week_6,week_7,week_8,week_9,week_10,week_11,week_12,week_13,week_14,week_15
0,"[westland, verlinde, sociaaldemocraten, vught, goirle, michel, uitspraak, werner, pgb, hellendoorn]","[partij, we, vvd, hoekstra, bbb, jaar, wel, cda, mensen, nieuwe]","[hoogtanders, bromet, loes, criminele, hulpverleners, roermond, recidive, arbeidsmigranten, latif, obl]","[museum, brug, cobra, raat, berends, staaij, touwen, steketee, tong, amstelveen]","[we, wel, kabinet, kamer, jaar, partij, partijen, vvd, omtzigt, euro]","[mos, spreidingswet, 50plus, hooft, burg, gemeentebestuur, aldi, herik, slob, vrijspraak]","[meijer, icar, vries, nur, eerlijke, kosterman, dijk, hannema, vitens, omroep]","[whisky, golfbaan, belastingvoordelen, marathon, keurmerk, opmerking, gedrag, zwabberen, pkn, nooijen]","[boeren, veestapel, rotonde, boer, pierik, woontorens, holman, winther, cdaplan, lto]","[omtzigt, we, partij, nsc, mensen, wel, partijen, jaar, kamer, vvd]","[bolkestein, sahla, toyota, elektrische, afkortingen, bontenbal, dorpsstraat, muzenval, erf, minibieb]","[longontsteking, boomsma, pvdagl, raadslid, sluis, marie, claire, vestingstad, minuten, ettenleur]","[we, wel, partij, omtzigt, vvd, mensen, partijen, zegt, wilders, timmermans]","[ijssel, zaanstad, castricum, middendelfland, nederbetuwe, uitgeest, alkmaar, katwijk, echtsusteren, diemen]","[bovens, samen1, thomas, gulden, apeldoorn, vooraf, overvallen, sahar, theo, cover]"
1,"[omtzigt, partij, we, jaar, kamer, wel, vvd, tweede, nieuwe, cda]","[pride, ijburg, tubbergen, evenement, yigits, lochtenberg, veenendaal, behoefte, geesteren, walk]","[tvdb, sk, white, brug, zuidoost, middendelfland, wijnschenk, grootouders, kuipers, vluchtelingenwerk]","[wilders, bruins, hoekstra, geert, asiel, sociaal, baudet, armoede, pvvleider, verkiezingsprogramma]","[willing, opleiding, simon, zuidholland, vismarkt, osdorp, polder, zuidhollandse, overdiepse, ozkaya]","[ouwehand, ns, parkeren, spits, wolf, grensoverschrijdend, heijnen, kernenergie, tunnelbak, horst]","[verhoging, ouwehand, kabinet, plannen, minimumloon, beek, sluis, kamer, kaag, partijbestuur]","[bbb, synagoge, azc, vermeer, plas, hilvarenbeek, waterpark, zamzam, veerse, hooge]","[baudet, vader, baarle, woningcorporaties, volgers, 2022, arnhem, braal, zwartewaal, platform]","[baudet, rutte, israel, arbeidsmigranten, politie, yesilgoz, premier, hamas, gent, timmermans]","[nissewaard, meter, pepijn, hendriks, bente, lahlah, hermans, minses, politie, zonnepark]","[rotterdam, hap, theater, pvdagl, kap, landbouwakkoord, ov, henri, kik, tibben]","[meta, advertenties, kennismigranten, microtargeting, bernheze, ella, westland, filmpje, boszhard, gijsbrecht]","[lochem, venray, roermond, dantumadiel, gemertbakel, noardeastfryslan, lansingerland, sintmichielsgestel, bilt, oirschot]","[uslu, corendon, staatssecretaris, gunay, demissionair, schreinemacher, wetenschap, demissionaire, cultuur, bedrijf]"
2,"[gundogan, onderwijs, zonnepanelen, waterschap, pieper, waarvan, wiersma, dijkgraaf, zelfstandig, wijk]","[koopmans, inkomens, vicepremier, woningen, brekelmans, prinsjesdag, miljard, bernheze, hogere, boek]","[burgerschap, immigratie, sportvissers, vissen, grip, sibbel, immigranten, afvalfonds, cabaretier, hengelsport]","[meijeren, candan, forum, aerden, geweld, overheid, opruiing, uitspraken, yesim, bamenga]","[bruls, hair, hoera, burg, broersen, rijswijk, minderjarige, azc, schollebos, kahya]","[burg, spreidingswet, pels, wetsvoorstel, eenzaamheid, huizenruil, thuiswerken, werkgever, brink, toos]","[democratie, ratten, directe, wind, sport, nick, 10000, langstraat, fierens, doneerde]","[korhoen, camping, zwol, groenafval, onteigening, grond, maquette, breda, twh, windmolens]","[voerman, vliegtaks, luchtvaart, eindhoven, sector, debatten, verkiezingsposter, asscher, agt, kanne]","[baarle, leus, the, palestine, sea, to, river, free, will, gepensioneerden]","[bolkestein, sahla, palmen, advocaten, frits, rongen, koffeman, levenslan